This is the radial basis function network implementation ( using the hybrid approach/ clustering based) and applied to an imbalanced train and test data set for credit scoring after data cleaning and preprocessing

Hyperparameters to control in this code :

nc= number of centers

optimizer

epochs

batch size

threshold

dropout

Packages import 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import *
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import explained_variance_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
import keras.backend as K
from keras.layers.core import Reshape
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.neural_network import MLPClassifier
from scipy.spatial import distance
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
from scipy.spatial import distance
from keras.utils import np_utils
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
import keras

Data import 

In [ ]:
#Train
path=r'C:/Users/zZiZzou_Tech/Desktop/engineerinternship/datascoring-train.csv'
train= pd.read_csv(path,sep=',')

In [ ]:
train.drop(train.columns[[0]], axis=1, inplace=True)

In [ ]:
train.shape

In [ ]:
train['Y'].value_counts()

In [ ]:
#test
path=r'C:/Users/zZiZzou_Tech/Desktop/engineerinternship/datascoring-test.csv'
test = pd.read_csv(path,sep=',')
test.shape

In [ ]:
test.drop(test.columns[[0]], axis=1, inplace=True)

In [ ]:
test.index = range(55087,83464)
test.shape

In [ ]:
test['Y'].value_counts()

In [ ]:
data=pd.concat([train,test])
data.head()

We have imbalanced data

# Data preprocessing

In [ ]:
AGE=list()
MONTH=list()
for i in range (len(data.index)):
    birth_date=str(data.NAISS[i])
    credit_date=str(data.DATDBL[i])
    AGE.append(int('20'+credit_date[(len(credit_date)-2):len(credit_date)])- int(birth_date[0:4]))
    if len(credit_date)==5:
       MONTH.append(int(credit_date[1:3]))
    else:
       MONTH.append(int(credit_date[2:4]))  
del data['NAISS']
del data['DATDBL']
data.insert(1,'AGE',AGE)
data.insert(2,'MONTH',MONTH) 

In [ ]:
data.head()

Spliting data into categorial and non categorial features

In [ ]:
non_categorial_features = ['NBENF',
                          'SALR',
                          'AGE',
                          'RMBMNT',
                          'CRDMNT',
                          'NBCPTE',
                          'BICCPENC',
                          'BICRIENC',
                          'NBDEB',
                          'NBCRD',
                          'SLDANN']

categorial_features=['SEX',
                    'AECO',
                    'SCPRO',
                    'PROF',
                    'ETCV',
                    'AGC',
                    'MONTH']


Scaling ( standard scaler) of the non categorial features

In [ ]:
data_quanti=data.loc[:,non_categorial_features]
scaler = StandardScaler(with_mean=True, with_std=True)
dataquanti_sc =scaler.fit_transform(data_quanti)


dataquanti=pd.DataFrame(dataquanti_sc)
dataquanti.columns= ['NBENF',
                          'SALR',
                          'AGE',
                          'RMBMNT',
                          'CRDMNT',
                          'NBCPTE',
                          'BICCPENC',
                          'BICRIENC',
                          'NBDEB',
                          'NBCRD',
                          'SLDANN']

Hot encoding for the categorial features

In [ ]:
data.shape

In [ ]:
data_quali=data.loc[:,categorial_features]
data_quali.shape

In [ ]:
for i in list(data_quali):
    data_quali[i] = data_quali[i].astype('category')
dataquali=pd.get_dummies(data_quali, sparse=True)

In [ ]:
dataquali.shape

## New data transformed

In [ ]:
newdata=pd.concat([data['Y'] ,dataquanti,dataquali],axis=1)
newdata.shape

## Respliting train and test

In [ ]:
train=newdata[:len(train)]
test=newdata[len(train):len(test)+len(train)]
test=test.reset_index(drop=True)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
X_train = train.iloc[:,1:] 
y_train=train['Y']

X_test = test.iloc[:,1:] 
y_test=test['Y']


In [ ]:
y_train.shape

In [ ]:
y_test.shape

# Radial Basis Function Network Implementation

In [ ]:
# Gaussian function

def RBFunction(r, x,c ):
    return( np.exp(- (distance.euclidean(x, c))**2/r**2))

# Computing radial/spread

def radial (x,c ):
    r=[]
    for i in range (c.shape[0]):
        k=[]
        for j in range(x.shape[0]) :
            k.append(distance.euclidean(x.iloc[j,:], c[i])) 
        m=(sum(k)/len(k))
        r.append(m)
    return(r)

# compute the centers using Kmeans

def centers (data,n):
    kmeans = KMeans(n_clusters=n).fit(data )
    return(kmeans.cluster_centers_)     

#Preparing the RBF network

def RBF (centers,radials, X): 
    s=[[]for k in range (centers.shape[0])]
    
    for i in range (centers.shape[0]):
        for j in range (X.shape[0]):
            X=pd.DataFrame(X)
            s[i] .append(RBFunction(radials[i] ,X.iloc[j,:],centers[i]))
    s=pd.DataFrame(s)
    return(s.transpose()) 
 


In [ ]:
train.Y.value_counts()

In [ ]:
test.Y.value_counts()

## First step: applying clustering  

In [ ]:
# Hyperparameter: number of centers/units of the hidden layer
nc= 40000
#Train
c=centers(X_train , nc)
r=radial(X_train,c)
inputs_train= pd.DataFrame(RBF(c,r,X_train))


#Test
inputs_t= pd.DataFrame(RBF(c,r,X_test))


In [ ]:
inputs_train.shape

In [ ]:
inputs_t.shape

# Using Sklearn package (MLPClassifier) to implement the second part of the RBFNetwork

In [ ]:
m1=MLPClassifier(activation='logistic' ,solver='lbfgs',momentum=0.9)

In [ ]:
mod1=m1.fit(inputs_train,y_train)

Assessing the performance of the model using different metrics

In [ ]:
#Train
out=m1.predict(inputs_train)
#accuracy 
acc1=m1.score(inputs_train,y_train)
#roc auc
auc1=roc_auc_score(y_train,out)
#precision
prec1=precision_score(y_train ,out)
#precision-recall auc
pr1=average_precision_score(y_train ,out)
#Recall
rec1=recall_score(y_train, out)
#F1 score
from sklearn.metrics import f1_score
f11=f1_score(y_train ,out)
m=[acc1,auc1,pr1,prec1,rec1,f11]
n=['Accuracy','ROC-auc','PR-auc','Precision','Recall','F1-score']
performance=pd.DataFrame([n,m])
performance

In [ ]:
#Test
out=m1.predict(inputs_t)
#accuracy 
acc1=m1.score(inputs_t,y_test)
#roc auc
auc1=roc_auc_score(y_test,out)
#pr-auc 
pr1=average_precision_score(y_test ,out)
#precision
rec1=precision_score(y_test,out)
#Recall
rec1=recall_score(y_test, out)
#F1 score
f11=f1_score(y_test ,out)
m=[acc1,auc1,pr1,prec1,rec1,f11]
n=['Accuracy','ROC-auc','PR-auc','Precision','Recall','F1-score']
performance=pd.DataFrame([n,m])
performance

In [ ]:
#roc curve:
fpr,tpr,_=roc_curve(y_test, out)
plt.plot(fpr,tpr)
plt.show()

In [ ]:
# Precision recall curve:
pr,rec,_=precision_recall_curve(y_test,probas_pred=out)
plt.plot(rec,pr)
plt.show()

In [ ]:
#Confusion matrix:
cm=pd.DataFrame(confusion_matrix(y_test,out))
cm

# Using Kears package  to implement the second part of the RBFNetwork

In [ ]:
#function used to modify threshold 
def out (v,s):
    c= []
    for i in range(v.shape[0]):
        if output[i,1]>s:
           c.append(1)
        else:
           c.append(0)
    return(c)

In [ ]:
#cost sensitive classification
class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

In [ ]:
inputs_train_arr=np.array(inputs_train)
inputs_t_arr=np.array(inputs_t)
y_train_arr=np.array(y_train)
y_t_arr=np.array(y_test)

dimof_input = inputs_train_arr.shape[1]

y_cat = np_utils.to_categorical(y_train_arr,2)
dimof_output_cat=y_cat.shape[1]

y_t_cat=np_utils.to_categorical(y_t_arr,2)
y_t_cat.shape

In [ ]:
#Hyperparameters

batch_size = 100
dropout = 0.2
epochs = 3000
verbose = 1
threshold=0.2 

## Set model


The different optimizers that can be used:
optimizers=['sgd','adam','rmsprop','adagrad','adamax','nadam']    

In [ ]:
def build_model(dimof_input,dimof_output):
    model = Sequential()
    model.add(Dense(dimof_output, input_dim=dimof_input))
    model.add(Dropout(dropout))
    model.add(Dense(2))
    model.add(Dropout(dropout))
    model.add(Dense(dimof_output, init=keras.initializers.RandomNormal(mean=0.0, stddev=0.05), activation='sigmoid'))
    return model

In [ ]:
model=build_model(dimof_input,dimof_output_cat)
model.compile(loss='binary_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(inputs_train_arr,y_cat, batch_size=batch_size,class_weight=class_weight, epochs=epochs,verbose=verbose)


In [ ]:
#Train
output=model.predict(inputs_train_arr)
output_t=out(output,threshold)
#accuracy
score, acc = model.evaluate(inputs_train_arr, y_cat)
#roc auc
auc1=roc_auc_score(y_train_arr ,output_t)
#precision
prec1=precision_score(y_train_arr ,output_t)
#precision-recall auc
pr1=average_precision_score(y_train_arr ,output_t)
#Recall
rec1=recall_score(y_train_arr, output_t)
#F1 score
f11=f1_score(y_train_arr ,output_t)
m=[acc,auc1,pr1,prec1,rec1,f11]
n=['Accuracy','ROC-auc','PR-auc','Precision','Recall','F1-score']
performance=pd.DataFrame([n,m])
performance

In [ ]:
#Test
output=model.predict(inputs_t_arr)
output_t=out(output,threshold)
#accuracy
score, acc = model.evaluate(inputs_t_arr, y_t_cat)
#roc auc
auc1=roc_auc_score(y_t_arr ,output_t)
#pr auc
pr1=average_precision_score(y_t_arr ,output_t)
#precision
prec1=precision_score(y_t_arr ,output_t)
#recall
rec1=recall_score(y_t_arr, output_t)
#f1score
f11=f1_score(y_t_arr ,output_t)

m=[acc,auc1,pr1,prec1,rec1,f11]
n=['Accuracy','ROC-auc','PR-auc','Precision','Recall','F1-score']

performance=pd.DataFrame([n,m])
performance

In [ ]:
#roc curve:
fpr,tpr,_=roc_curve(y_test, output[:,1])
plt.plot(fpr,tpr)
plt.show()

In [ ]:
# Precision recall curve:
pr,rec,_=precision_recall_curve(y_test , probas_pred=output[:,1])
plt.plot(rec,pr)
plt.show()

In [ ]:
#Confusion matrix:
cm=pd.DataFrame(confusion_matrix(y_test,output_t))
cm